In [1]:
import pandas as pd
import requests

In [2]:
CMC_PRO_API_KEY= "31d0c093-d32a-4d4f-9ea4-b7f894edc76b"

In [3]:
coin_names = pd.read_csv("coinnames.csv")

In [4]:
coin_names.head(10)

,Ticker
0,BTC
1,ETH
2,BCH
3,XRP
4,DASH
5,LTC
6,ETC
7,ADA
8,MIOTA
9,XLM


### CoinMarketCap API Implementation

In [5]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

def cmc_api(url, parameters):
    headers = {
      'Accepts': 'application/json',
      'Accept-Encoding': 'deflate, gzip',
      'X-CMC_PRO_API_KEY': f'{CMC_PRO_API_KEY}',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

Example Parameter and url to pass

In [6]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
parameters = {
  'id':'1',
  'convert':'USD',
  'aux': 'num_market_pairs,cmc_rank,tags,platform,max_supply,circulating_supply,total_supply,market_cap_by_total_supply'
}

Map_data includes CMC IDs, symbols, and names for all cryptocurrencies

In [7]:
map_data

NameError: name 'map_data' is not defined

Example of a cryptocurrency quote for Bitcoin:

In [ ]:
btc_data["data"]

{'1': {'id': 1,
  'name': 'Bitcoin',
  'symbol': 'BTC',
  'slug': 'bitcoin',
  'num_market_pairs': 8291,
  'tags': ['mineable',
   'pow',
   'sha-256',
   'store-of-value',
   'state-channels',
   'coinbase-ventures-portfolio',
   'three-arrows-capital-portfolio',
   'polychain-capital-portfolio',
   'binance-labs-portfolio',
   'arrington-xrp-capital',
   'blockchain-capital-portfolio',
   'boostvc-portfolio',
   'cms-holdings-portfolio',
   'dcg-portfolio',
   'dragonfly-capital-portfolio',
   'electric-capital-portfolio',
   'fabric-ventures-portfolio',
   'framework-ventures',
   'galaxy-digital-portfolio',
   'huobi-capital',
   'alameda-research-portfolio',
   'a16z-portfolio',
   '1confirmation-portfolio',
   'winklevoss-capital',
   'usv-portfolio',
   'placeholder-ventures-portfolio',
   'pantera-capital-portfolio',
   'multicoin-capital-portfolio',
   'paradigm-xzy-screener'],
  'max_supply': 21000000,
  'circulating_supply': 18865956,
  'total_supply': 18865956,
  'platform'

# Get the ids for the coins available in EToro from map_data

### Get relevant fields ("id", "symbol", "name") from map_data 

In [ ]:
map_data_copy = []
map_data_copy += [map_data["data"][i][key] for i in range(len(map_data["data"])) for key in ["id","symbol","name"]]

### Group data in lists of lists 

In [ ]:
new_data = []
new_data += [map_data_copy[i:i+3] for i in range(0, len(map_data_copy)-2, 3)]

In [ ]:
new_data[:10]

[[1, 'BTC', 'Bitcoin'],
 [2, 'LTC', 'Litecoin'],
 [3, 'NMC', 'Namecoin'],
 [4, 'TRC', 'Terracoin'],
 [5, 'PPC', 'Peercoin'],
 [6, 'NVC', 'Novacoin'],
 [8, 'FTC', 'Feathercoin'],
 [10, 'FRC', 'Freicoin'],
 [13, 'IXC', 'Ixcoin'],
 [16, 'WDC', 'WorldCoin']]

### Put this in a PandasDataframe for ease of use

In [ ]:
map_df = pd.DataFrame(new_data, columns=["id","Symbol","Name"])
map_df.head()

,id,Symbol,Name
0,1,BTC,Bitcoin
1,2,LTC,Litecoin
2,3,NMC,Namecoin
3,4,TRC,Terracoin
4,5,PPC,Peercoin


### Use the coin names contained in the coin_names list to get the ids for coins that are available on Etoro

In [ ]:
coin_names.rename(columns={"Ticker":"Symbol"}, inplace=True)
coin_names.head()

,Symbol
0,BTC
1,ETH
2,BCH
3,XRP
4,DASH


### Merge dataframes on the Symbol column

In [ ]:
merged_df = pd.merge(map_df, coin_names, on='Symbol')

In [ ]:
merged_df

,id,Symbol,Name
0,1,BTC,Bitcoin
1,2,LTC,Litecoin
2,52,XRP,XRP
3,74,DOGE,Dogecoin
4,131,DASH,Dash
5,512,XLM,Stellar
6,1027,ETH,Ethereum
7,1321,ETC,Ethereum Classic
8,1376,NEO,Neo
9,1437,ZEC,Zcash


### Get all ids into a single string

In [ ]:
id_list = [id for id in merged_df[["id"]].iloc[:,0]]

In [ ]:
id_string = ""
#get all ids in one string
for id in id_list:
    id_string += str(id) + ","
    
#remove last comma
id_string = id_string.strip(",")

### Call API with all relevant IDs

In [ ]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"

parameters = {
  'id':f'{id_string}',
  'convert':'USD',
  'aux': 'num_market_pairs,cmc_rank,tags,platform,max_supply,circulating_supply,total_supply,market_cap_by_total_supply'
}

coin_quotes = cmc_api(url, parameters)

In [ ]:
coin_quotes

{'status': {'timestamp': '2021-11-07T15:29:37.886Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 13,
  'credit_count': 1,
  'notice': None},
 'data': {'1': {'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'slug': 'bitcoin',
   'num_market_pairs': 8291,
   'tags': ['mineable',
    'pow',
    'sha-256',
    'store-of-value',
    'state-channels',
    'coinbase-ventures-portfolio',
    'three-arrows-capital-portfolio',
    'polychain-capital-portfolio',
    'binance-labs-portfolio',
    'arrington-xrp-capital',
    'blockchain-capital-portfolio',
    'boostvc-portfolio',
    'cms-holdings-portfolio',
    'dcg-portfolio',
    'dragonfly-capital-portfolio',
    'electric-capital-portfolio',
    'fabric-ventures-portfolio',
    'framework-ventures',
    'galaxy-digital-portfolio',
    'huobi-capital',
    'alameda-research-portfolio',
    'a16z-portfolio',
    '1confirmation-portfolio',
    'winklevoss-capital',
    'usv-portfolio',
    'placeholder-ventures-portfolio',
  

### Extract relevant data from coin_quotes 

In [ ]:
#get a list of dictionaries
coin_quotes_list = [value for value in coin_quotes["data"].values()]

In [ ]:
coin_quotes_list
len(coin_quotes_list)

40

Get list of relevant fields from the dicts

In [ ]:
from operator import itemgetter
temp = [itemgetter("id", "name", "symbol", "quote")(coin_quotes_list[i]) for i in range(len(coin_quotes_list))]
temp

[(1,
  'Bitcoin',
  'BTC',
  {'USD': {'price': 61817.78239076061,
    'volume_24h': 25038723279.23909,
    'volume_change_24h': -17.0438,
    'percent_change_1h': -0.39430395,
    'percent_change_24h': 1.78396473,
    'percent_change_7d': 1.77794044,
    'percent_change_30d': 13.25700736,
    'percent_change_60d': 34.02405551,
    'percent_change_90d': 34.53747557,
    'market_cap': 1166282842399.5542,
    'market_cap_dominance': 42.204,
    'fully_diluted_market_cap': 1298173430205.97,
    'market_cap_by_total_supply': 1166282842399.5542,
    'last_updated': '2021-11-07T15:28:02.000Z'}}),
 (2,
  'Litecoin',
  'LTC',
  {'USD': {'price': 199.67508544626386,
    'volume_24h': 1354702269.6045868,
    'volume_change_24h': -14.8736,
    'percent_change_1h': -0.10227462,
    'percent_change_24h': 2.41245627,
    'percent_change_7d': 5.14942138,
    'percent_change_30d': 9.70080173,
    'percent_change_60d': 10.28473558,
    'percent_change_90d': 21.87094183,
    'market_cap': 13761623386.007

Extract market cap data and store in temp lists

In [ ]:
market_caps = [temp[i][3]["USD"]["market_cap"] for i in range(len(temp))]
market_cap_dominances = [temp[i][3]["USD"]["market_cap_dominance"] for i in range(len(temp))]
market_caps

[1166282842399.5542,
 13761623386.007582,
 57011237096.0785,
 34971437132.85317,
 2010621126.8396711,
 8774644306.580523,
 542390676446.66644,
 7026421414.476563,
 3205457070.8473043,
 2098822300.6313105,
 2924573588.2783346,
 1485193112.557301,
 3745290709.01238,
 4308977701.839139,
 11243503223.36315,
 110029505013.77098,
 7435504664.020182,
 5177320223.807516,
 14907439403.85547,
 67030627627.43331,
 5498645614.734127,
 2747911379.8077693,
 7472220033.086289,
 4549.121203026612,
 8027940798.565071,
 13001230034.006084,
 11535298249.412134,
 13222.435479059543,
 76307722031.62643,
 2170438898.4377985,
 0,
 1254806710.6548908,
 1803520062.6289904,
 52408538150.912926,
 5016942287.536716,
 15808649094.064651,
 4257207875.1517262,
 821914705.4863534,
 0,
 0]

## Store these all in a Pandas Table 

Add these two lists to existing dataframe:merged_df

In [ ]:
merged_df["Market Cap"] = market_caps
merged_df["Market Cap Share"] = market_cap_dominances
merged_df.head()

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1.166283e+12,42.2040,2.501947
1,2,LTC,Litecoin,1.376162e+10,0.4978,1.210857
2,52,XRP,XRP,5.701124e+10,2.0632,5.018561
3,74,DOGE,Dogecoin,3.497144e+10,1.2656,3.078466
4,131,DASH,Dash,2.010621e+09,0.0728,0.177080


In [ ]:
merged_df.sort_values(by="Market Cap")

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
39,8104,1INCH,1inch Network,0.000000e+00,0.0000,0.000000
30,7083,UNI,Uniswap,0.000000e+00,0.0000,0.000000
38,7278,AAVE,Aave,0.000000e+00,0.0000,0.000000
23,2280,FIL,Filecoin,4.549121e+03,0.0000,0.000000
27,3890,MATIC,Polygon,1.322244e+04,0.0000,0.000000
37,6636,DOT,Polkadot,8.219147e+08,0.0297,0.072243
31,5426,SOL,Solana,1.254807e+09,0.0454,0.110432
11,1697,BAT,Basic Attention Token,1.485193e+09,0.0537,0.130621
32,12003,SOL,SOL RUNE - Rune.Game,1.803520e+09,0.0653,0.158837
4,131,DASH,Dash,2.010621e+09,0.0728,0.177080


Convert to billions

In [ ]:
#merged_df["Market Cap"] = merged_df["Market Cap"]/10**9
merged_df.head()

,id,Symbol,Name,Market Cap,Market Cap Share
0,1,BTC,Bitcoin,1166.282842,42.2040
1,2,LTC,Litecoin,13.761623,0.4978
2,52,XRP,XRP,57.011237,2.0632
3,74,DOGE,Dogecoin,34.971437,1.2656
4,131,DASH,Dash,2.010621,0.0728


## Save the dataframe 

In [ ]:
merged_df.to_csv("CoinMarketCaps07.11.21.csv")

# Calculations for Equal weight portfolio

In [ ]:
merged_df["Market Cap Share"].sum()

82.2868

In [ ]:
merged_df["Market Cap Share"] = merged_df["Market Cap Share"]/82.2868*100
merged_df.head()

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,1.600000
1,2,LTC,Litecoin,13.761623,0.604957,14.197743
2,52,XRP,XRP,57.011237,2.507328,58.844482
3,74,DOGE,Dogecoin,34.971437,1.538035,36.096150
4,131,DASH,Dash,2.010621,0.088471,2.076327


In [ ]:
merged_df["Market Cap Share-Modified"] = merged_df["Market Cap Share"]
merged_df.head()

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,51.288907
1,2,LTC,Litecoin,13.761623,0.604957,0.604957
2,52,XRP,XRP,57.011237,2.507328,2.507328
3,74,DOGE,Dogecoin,34.971437,1.538035,1.538035
4,131,DASH,Dash,2.010621,0.088471,0.088471


### Limit Bitcoin exposure and recalculate 

In [ ]:
merged_df.iloc[0,5] = 1.25
merged_df.head(1)

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,1.25


In [ ]:
merged_df["Market Cap Share-Modified"].sum()

49.96109339529548

In [ ]:
merged_df.iloc[:,5] = merged_df.iloc[:,5] / merged_df["Market Cap Share-Modified"].sum() * 100
merged_df[(merged_df["Market Cap Share-Modified"] > 2)]

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,2.501947
2,52,XRP,XRP,57.011237,2.507328,5.018561
3,74,DOGE,Dogecoin,34.971437,1.538035,3.078466
6,1027,ETH,Ethereum,542.390676,23.852793,47.742736
15,1839,BNB,Binance Coin,110.029505,4.838808,9.685152
19,1975,LINK,Chainlink,67.030628,2.947982,5.900555
28,4030,ALGO,Algorand,76.307722,3.355945,6.717117
33,5711,GRT,Golden Ratio Token,52.408538,2.303772,4.611131


## Check which coins, without manipulating bitcoin, would reach threshold 

In [ ]:
merged_df[(merged_df["Market Cap Share"] > 2)]

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,2.501947
2,52,XRP,XRP,57.011237,2.507328,5.018561
6,1027,ETH,Ethereum,542.390676,23.852793,47.742736
15,1839,BNB,Binance Coin,110.029505,4.838808,9.685152
19,1975,LINK,Chainlink,67.030628,2.947982,5.900555
28,4030,ALGO,Algorand,76.307722,3.355945,6.717117
33,5711,GRT,Golden Ratio Token,52.408538,2.303772,4.611131


For a 1000USD Portfolio with minimum purchase size of 25$, minimum share is 2.5%

In [ ]:
larger_coins = merged_df[(merged_df["Market Cap Share-Modified"] > 1)]
larger_coins

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,15.695854
2,52,XRP,XRP,57.011237,2.507328,3.935465
3,74,DOGE,Dogecoin,34.971437,1.538035,2.414078
6,1027,ETH,Ethereum,542.390676,23.852793,37.438995
15,1839,BNB,Binance Coin,110.029505,4.838808,7.594922
18,12712,MANA,Genesis Mana,14.907439,0.655634,1.029073
19,1975,LINK,Chainlink,67.030628,2.947982,4.627109
28,4030,ALGO,Algorand,76.307722,3.355945,5.267443
33,5711,GRT,Golden Ratio Token,52.408538,2.303772,3.615966
35,5864,YFI,yearn.finance,15.808649,0.694887,1.090684


In [ ]:
larger_coins["Market Cap Share-Modified"].sum()

82.7095885195501

## Sort original list for reference 

In [ ]:
merged_df.sort_values(by="Market Cap",ascending=False)

,id,Symbol,Name,Market Cap,Market Cap Share,Market Cap Share-Modified
0,1,BTC,Bitcoin,1166.282842,51.288907,2.501947
6,1027,ETH,Ethereum,542.390676,23.852793,47.742736
15,1839,BNB,Binance Coin,110.029505,4.838808,9.685152
28,4030,ALGO,Algorand,76.307722,3.355945,6.717117
19,1975,LINK,Chainlink,67.030628,2.947982,5.900555
2,52,XRP,XRP,57.011237,2.507328,5.018561
33,5711,GRT,Golden Ratio Token,52.408538,2.303772,4.611131
3,74,DOGE,Dogecoin,34.971437,1.538035,3.078466
35,5864,YFI,yearn.finance,15.808649,0.694887,1.390856
18,12712,MANA,Genesis Mana,14.907439,0.655634,1.312289


# Define a function to easily let you manipulate market cap share 

## TODO:
There is an error associated with the market caps of some coins, which falsely appear to be zero- fix these
Then write a funciton to customize your portfolio by setting max exposure rules

In [ ]:
def portfolio_customizer(table,)